In [1]:
import torch
from transformers import AutoTokenizer, ElectraForSequenceClassification, TrainingArguments, Trainer, AutoModel, AutoModelForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd
from Dataset import CustomDataset

/home/mingyu13/바탕화면/ToU-Project-Artificial-Intelligence-Archived/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv("dataset_path", sep="\t")
df.info()

### sep를 tap으로 하고 content하고 label을 분리하면 content의 끝문자열에 특수문자가 있는 경우 정상적으로 sep가 안될 수 있다. df.info를 통해 확인하고 content의 데이터수와 label의 데이터수가 다르다면 다음 코드를 실행해준다.

In [ ]:
null_idx = df[df.lable.isnull()].index

# lable 은 content의 가장 끝 문자열로 설정.
df.loc[null_idx, "lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정.
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[:-2])
df = df.astype({"lable":"int"})




In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

test_data.drop_duplicates(subset=["content"], inplace= True)



tokenized_test_sentences = tokenizer(
    list(test_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

# print(tokenized_test_sentences[0])
# print(tokenized_test_sentences[0].tokens)
# print(tokenized_test_sentences[0].ids)
# print(tokenized_test_sentences[0].attention_mask)


test_label = test_data["lable"].values

test_dataset = CustomDataset(tokenized_test_sentences, test_label)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained('fastival_model', num_labels=2)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")




In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,                         # 학습하고자하는  Transformers model
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)